In [5]:

# ============ 1. ИМПОРТ БИБЛИОТЕК ============
import asyncio
import aiohttp
import json
from datetime import datetime, timedelta
import os
import sys
import hashlib
import numpy as np
import subprocess
import glob
import time
import io

# Отключаем предупреждение о symlinks на Windows
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

# Функция для установки библиотек
def install_packages():
    required = ['aiohttp', 'numpy', 'scikit-learn']
    print(" Проверяем библиотеки...")
    for package in required:
        try:
            __import__(package.replace('-', '_'))
        except ImportError:
            print(f" Устанавливаем {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

install_packages()

# ============ 2. БЕЗОПАСНЫЙ КЛАСС ДЛЯ РАБОТЫ С ИЗОБРАЖЕНИЯМИ ============
import requests
from PIL import Image, ImageFile
import warnings

# Отключаем предупреждение о декомпрессионной бомбе
warnings.filterwarnings("ignore", category=Image.DecompressionBombWarning)

# Увеличиваем лимит на размер изображения
Image.MAX_IMAGE_PIXELS = 200000000  # 200 миллионов пикселей

# Разрешаем загрузку усеченных изображений
ImageFile.LOAD_TRUNCATED_IMAGES = True

class SafeImageDownloader:
    """Безопасный загрузчик изображений с защитой от больших файлов"""
    
    @staticmethod
    def download_and_resize(image_url, max_size=(800, 600), timeout=10):
        """Скачивание и безопасное изменение размера изображения"""
        try:
            # Сначала проверяем размер заголовков
            head_response = requests.head(image_url, timeout=5, allow_redirects=True)
            content_length = head_response.headers.get('content-length')
            
            if content_length:
                file_size = int(content_length)
                # Если файл больше 10MB, пропускаем
                if file_size > 10 * 1024 * 1024:
                    print(f"      Изображение слишком большое: {file_size//1024}KB, пропускаем")
                    return None
            
            # Скачиваем изображение
            response = requests.get(image_url, timeout=timeout, stream=True)
            response.raise_for_status()
            
            # Проверяем размер в памяти
            content = response.content
            if len(content) > 5 * 1024 * 1024:  # Больше 5MB
                print(f"      Изображение в памяти слишком большое: {len(content)//1024}KB")
                return None
            
            # Открываем изображение с ограничением
            img = Image.open(io.BytesIO(content))
            
            # Проверяем размеры изображения
            if img.width * img.height > 10000000:  # Больше 10 миллионов пикселей
                print(f"      Изображение слишком большое: {img.width}x{img.height}")
                # Все равно уменьшаем, но с предупреждением
                img.thumbnail((400, 300), Image.Resampling.LANCZOS)
            
            # Конвертируем в RGB если нужно
            if img.mode in ('RGBA', 'LA', 'P'):
                img = img.convert('RGB')
            
            # Уменьшаем размер
            img.thumbnail(max_size, Image.Resampling.LANCZOS)
            
            # Сохраняем в буфер
            buffered = io.BytesIO()
            img.save(buffered, format="JPEG", quality=85, optimize=True)
            
            # Возвращаем URL и информацию о размере (не сохраняем само изображение)
            return {
                'url': image_url,
                'width': img.width,
                'height': img.height,
                'format': img.format or 'JPEG',
                'size_kb': len(buffered.getvalue()) // 1024
            }
            
        except Exception as e:
            print(f"      Ошибка загрузки изображения: {str(e)[:50]}...")
            return None

# ============ 3. УПРОЩЕННАЯ МОДЕЛЬ ДЛЯ ЭМБЕДДИНГОВ ============
class SimpleEmbeddingModel:
    """Упрощенная модель для создания эмбеддингов"""
    
    def __init__(self, vector_size=192):
        self.vector_size = vector_size
        print(f" Используем упрощенную модель (размер вектора: {vector_size})")
        
    def encode(self, text):
        """Создание псевдо-эмбеддинга на основе хэша текста"""
        if not text:
            return np.zeros(self.vector_size)
        
        text_hash = hashlib.sha256(text.encode()).hexdigest()
        seed = int(text_hash[:8], 16)
        np.random.seed(seed)
        vector = np.random.randn(self.vector_size)
        
        norm = np.linalg.norm(vector)
        if norm > 0:
            vector = vector / norm
            
        return vector

# ============ 4. КЛАСС ДЛЯ ПАРСИНГА KUDAGO ============
class RealKudaGoParser2025:
    """Парсер РЕАЛЬНЫХ событий 2024-2025 года с KudaGo API"""
    
    def __init__(self, city='spb'):
        self.base_url = "https://kudago.com/public-api/v1.4"
        self.city = city
        self.cities = {
            'spb': {'name': 'Санкт-Петербург', 'timezone': 'Europe/Moscow'},
            'msk': {'name': 'Москва', 'timezone': 'Europe/Moscow'}
        }
    
    async def fetch_events_by_category(self, session, category_code, category_name):
        """Загрузка событий по категории"""
        print(f" Загружаем {category_name}...")
        
        # Берем события на ближайший год (от текущей даты до конца 2025)
        current_date = datetime.now()
        # Форматируем дату начала (сегодня)
        start_date_str = current_date.strftime('%Y-%m-%d')
        # Форматируем дату окончания (конец 2025 года)
        end_2025_str = '2025-12-31'
        
        params = {
            'categories': category_code,
            'location': self.city,
            'page_size': 100,  # Увеличили для большего охвата
            'actual_since': start_date_str,  # Формат: '2025-12-10'
            'actual_until': end_2025_str,    # Формат: '2025-12-31'
            'fields': ('id,title,description,dates,price,place,images,'
                      'site_url,tags,age_restriction,participants'),
            'expand': 'place',
            'text_format': 'text'  # Получаем чистый текст
        }
        
        try:
            # Ключевое изменение: отключаем автоматический вызов исключения
            # для ручного чтения тела ответа при ошибке 400
            async with session.get(
                f"{self.base_url}/events/",
                params=params,
                timeout=aiohttp.ClientTimeout(total=60),
                raise_for_status=False  # Не выбрасывать исключение автоматически
            ) as response:
                
                # Читаем тело ответа ВНЕ зависимости от статуса
                response_body = await response.text()
                
                # Теперь проверяем статус вручную
                if response.status == 200:
                    data = await response.json()
                    events = data.get('results', [])
                    
                    # Фильтруем события, чтобы были в будущем
                    filtered_events = []
                    for event in events:
                        if self._is_event_in_future(event):
                            filtered_events.append(event)
                    
                    print(f"     {len(filtered_events)} актуальных событий")
                    return filtered_events, category_name
                else:
                    # Подробное логирование при ошибке 400
                    print(f"     Ошибка HTTP {response.status} при запросе {category_name}")
                    print(f"     Тело ответа от сервера: {response_body[:500]}")  # Первые 500 символов
                    # Попробуйте также распарсить ответ как JSON, если он в таком формате
                    try:
                        error_json = json.loads(response_body)
                        print(f"     JSON ошибки: {error_json}")
                    except:
                        pass
                    return [], category_name
                    
        except asyncio.TimeoutError:
            print(f"     Таймаут при запросе {category_name}")
            return [], category_name
        except Exception as e:
            print(f"     Иная ошибка: {str(e)[:80]}...")
            return [], category_name
    
    def _is_event_in_future(self, event):
        """Проверка, что событие происходит в будущем"""
        dates = event.get('dates', [])
        if not dates:
            return False
        
        current_time = time.time()
        for date_info in dates:
            start = date_info.get('start')
            if start and start > current_time:
                return True
        return False
    
    def _process_event(self, event, category):
        """Обработка и форматирование события"""
        if not event.get('title'):
            return None
        
        # Обработка места проведения
        place_info = event.get('place', {})
        place_details = {}
        place_text = ""
        
        if isinstance(place_info, dict) and place_info:
            place_details = {
                'id': place_info.get('id'),
                'title': place_info.get('title', ''),
                'address': place_info.get('address', ''),
                'subway': place_info.get('subway', ''),
                'location': place_info.get('location', {}),
                'phone': place_info.get('phone', ''),
                'site_url': place_info.get('site_url', ''),
                'is_stub': place_info.get('is_stub', False)
            }
            
            # Формируем текстовое описание места
            place_parts = []
            if place_info.get('title'):
                place_parts.append(place_info['title'])
            if place_info.get('address'):
                place_parts.append(f"адрес: {place_info['address']}")
            if place_info.get('subway'):
                place_parts.append(f"метро: {place_info['subway']}")
            place_text = ", ".join(place_parts)
        
        # Обработка дат
        dates_info = []
        dates_text = []
        
        for date_item in event.get('dates', []):
            start = date_item.get('start')
            end = date_item.get('end')
            
            if start:
                try:
                    start_dt = datetime.fromtimestamp(start)
                    date_info = {
                        'start': start,
                        'end': end,
                        'start_datetime': start_dt.isoformat(),
                        'year': start_dt.year
                    }
                    
                    date_str = start_dt.strftime('%d.%m.%Y %H:%M')
                    if end:
                        end_dt = datetime.fromtimestamp(end)
                        date_str += f" - {end_dt.strftime('%d.%m.%Y %H:%M')}"
                    
                    dates_info.append(date_info)
                    dates_text.append(date_str)
                except:
                    continue
        
        if not dates_info:
            return None
        
        # Обработка цены
        price_info = event.get('price', 'Не указана')
        price_text = str(price_info)
        price_details = {}
        
        if isinstance(price_info, dict):
            price_details = price_info
            if price_info.get('is_free'):
                price_text = "Бесплатно"
            else:
                min_p = price_info.get('min', '')
                max_p = price_info.get('max', '')
                if min_p and max_p:
                    price_text = f"от {min_p} до {max_p} руб."
                elif min_p:
                    price_text = f"от {min_p} руб."
                elif max_p:
                    price_text = f"до {max_p} руб."
        
        # Обработка описания
        description = event.get('description', '')
        # Убираем HTML теги и лишние пробелы
        if description:
            import re
            description = re.sub(r'<[^>]+>', ' ', description)
            description = re.sub(r'\s+', ' ', description).strip()
        
        short_description = description[:250] + "..." if len(description) > 250 else description
        
        # Обработка изображений (только метаданные, не сами изображения)
        images_data = []
        original_images = event.get('images', [])
        
        for img in original_images[:2]:  # Берем только 2 изображения
            if isinstance(img, dict):
                image_info = {
                    'url': img.get('image', ''),
                    'thumbnails': {
                        'small': img.get('thumbnail', ''),
                        'medium': img.get('image', ''),  # оригинал как medium
                        'large': img.get('image', '')   # и как large
                    },
                    'source': 'kudago'
                }
                images_data.append(image_info)
        
        # Создание текста для эмбеддинга
        embedding_text = self._create_embedding_text(
            event.get('title', ''),
            description,
            category,
            place_text,
            event.get('tags', []),
            dates_text[0] if dates_text else ""
        )
        
        return {
            'id': event.get('id'),
            'title': event.get('title'),
            'description': description,
            'short_description': short_description,
            'category': category,
            'dates': dates_info,
            'dates_text': dates_text,
            'price': price_text,
            'price_details': price_details,
            'place': place_details,
            'place_text': place_text,
            'url': event.get('site_url', ''),
            'tags': event.get('tags', []),
            'images': images_data,
            'image_count': len(images_data),
            'age_restriction': event.get('age_restriction'),
            'participants': event.get('participants', []),
            'city': self.city,
            'city_name': self.cities.get(self.city, {}).get('name', self.city),
            'parsed_at': datetime.now().isoformat(),
            'embedding_text': embedding_text,
            'is_real_data': True,
            'source': 'kudago'
        }
    
    def _create_embedding_text(self, title, description, category, place, tags, date):
        """Создание текста для эмбеддинга"""
        parts = []
        if title:
            parts.append(f"Название: {title}")
        if description:
            parts.append(f"Описание: {description[:400]}")
        if category:
            parts.append(f"Категория: {category}")
        if place:
            parts.append(f"Место: {place}")
        if date:
            parts.append(f"Дата: {date}")
        if tags:
            parts.append(f"Теги: {', '.join(tags[:8])}")
        
        return " | ".join(parts)
    
    async def get_real_events(self):
        """Получение реальных актуальных событий"""
        city_name = self.cities.get(self.city, {}).get('name', self.city)
        print(f"\n  Парсим РЕАЛЬНЫЕ актуальные события для {city_name}")
        print("="*60)
        
        # Категории для парсинга (удалена неподдерживаемая категория 'performance')
        categories = [
            ('concert', 'Концерты'),
            ('theater', 'Театр'),
            ('exhibition', 'Выставки'),
            ('festival', 'Фестивали'),
            ('education', 'Образование'),
            ('party', 'Вечеринки'),
            ('tour', 'Экскурсии'),
            ('entertainment', 'Развлечения')  # Оставлены только поддерживаемые категории
        ]
        
        all_events = []
        
        async with aiohttp.ClientSession() as session:
            for cat_code, cat_name in categories:
                events, category_name = await self.fetch_events_by_category(session, cat_code, cat_name)
                
                if events:
                    print(f"   Обрабатываем {len(events)} событий категории {category_name}...")
                    
                    processed_count = 0
                    for event in events:
                        processed_event = self._process_event(event, category_name)
                        if processed_event:
                            all_events.append(processed_event)
                            processed_count += 1
                    
                    print(f"     Обработано {processed_count} событий")
                else:
                    print(f"     Нет событий в категории {category_name}")
                
                # Задержка для избежания блокировки
                await asyncio.sleep(1.5)
        
        print(f"\n Получено {len(all_events)} РЕАЛЬНЫХ событий для {city_name}")
        
        # Анализ дат событий
        if all_events:
            self._analyze_event_dates(all_events, city_name)
        
        return all_events
    
    def _analyze_event_dates(self, events, city_name):
        """Анализ дат событий"""
        current_year = datetime.now().year
        years_count = {}
        
        for event in events:
            dates = event.get('dates', [])
            if dates:
                for date_info in dates:
                    year = date_info.get('year')
                    if year:
                        years_count[year] = years_count.get(year, 0) + 1
        
        print(f"\n Анализ дат событий для {city_name}:")
        for year, count in sorted(years_count.items()):
            status = "ТЕКУЩИЙ" if year == current_year else "БУДУЩЕЕ" if year > current_year else "ПРОШЛОЕ"
            print(f"  {year}: {count} событий ({status})")
# ============ 5. КЛАСС ВЕКТОРНОЙ БАЗЫ ============
class RealVectorDatabase:
    """Векторная база данных для реальных событий"""
    
    def __init__(self, folder='real_vector_db'):
        self.folder = folder
        os.makedirs(folder, exist_ok=True)
        
        self.model = SimpleEmbeddingModel(vector_size=192)
        self.embedding_dim = 192
        print(" Используем модель для векторной БД")
    
    def create_embedding(self, text):
        """Создание векторного представления"""
        return self.model.encode(text).tolist()
    
    def save_events(self, events, filename=None):
        """Сохранение событий с эмбеддингами"""
        if not events:
            print(" Нет событий для сохранения")
            return None
        
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"real_events_{timestamp}.json"
        
        filepath = os.path.join(self.folder, filename)
        
        print(f" Создаем эмбеддинги для {len(events)} событий...")
        
        processed = []
        
        for i, event in enumerate(events):
            # Создаем эмбеддинг
            embedding_text = event.get('embedding_text', '')
            if not embedding_text:
                text_parts = []
                if event.get('title'):
                    text_parts.append(event['title'])
                if event.get('description'):
                    text_parts.append(event['description'][:300])
                if event.get('category'):
                    text_parts.append(event['category'])
                if event.get('place_text'):
                    text_parts.append(event['place_text'])
                embedding_text = " ".join(text_parts)
            
            embedding = self.create_embedding(embedding_text)
            
            event_data = {
                'id': event.get('id'),
                'title': event.get('title'),
                'category': event.get('category'),
                'city': event.get('city'),
                'city_name': event.get('city_name'),
                'price': event.get('price'),
                'place_text': event.get('place_text'),
                'dates_text': event.get('dates_text', []),
                'url': event.get('url'),
                'tags': event.get('tags', []),
                'images': event.get('images', []),
                'image_count': event.get('image_count', 0),
                'age_restriction': event.get('age_restriction'),
                'description': event.get('description', ''),
                'short_description': event.get('short_description', ''),
                'embedding': embedding,
                'embedding_text': embedding_text,
                'embedding_dim': self.embedding_dim,
                'parsed_at': event.get('parsed_at'),
                'source': event.get('source', 'kudago'),
                'is_real_data': event.get('is_real_data', True)
            }
            processed.append(event_data)
            
            if (i + 1) % 20 == 0:
                print(f"  Обработано {i + 1}/{len(events)}")
        
        # Метаданные
        metadata = {
            'total_events': len(processed),
            'created_at': datetime.now().isoformat(),
            'model_type': 'simple',
            'embedding_dim': self.embedding_dim,
            'cities': list(set(e['city'] for e in processed)),
            'categories': list(set(e['category'] for e in processed)),
            'has_images': any(e.get('image_count', 0) > 0 for e in processed),
            'total_images': sum(e.get('image_count', 0) for e in processed),
            'source': 'KudaGo API'
        }
        
        # Сохраняем в файл
        data = {'metadata': metadata, 'events': processed}
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"\n Данные сохранены: {filepath}")
        print(f" Статистика:")
        print(f"  • Событий: {metadata['total_events']}")
        print(f"  • Изображений: {metadata['total_images']}")
        print(f"  • Города: {', '.join(metadata['cities'])}")
        print(f"  • Размер файла: {os.path.getsize(filepath) // 1024} KB")
        
        return filepath
    
    def search_similar(self, query, top_k=5, city_filter=None):
        """Поиск похожих событий"""
        files = [f for f in os.listdir(self.folder) if f.endswith('.json')]
        if not files:
            print(" Нет файлов с данными")
            return []
        
        latest = sorted(files)[-1]
        filepath = os.path.join(self.folder, latest)
        
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        events = data.get('events', [])
        
        # Фильтрация
        if city_filter:
            events = [e for e in events if e['city'] == city_filter]
        
        if not events:
            return []
        
        # Эмбеддинг запроса
        query_embedding = self.create_embedding(query)
        
        # Вычисляем сходство
        results = []
        for event in events:
            event_embedding = np.array(event['embedding'])
            
            similarity = np.dot(query_embedding, event_embedding) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(event_embedding) + 1e-8
            )
            
            results.append({
                'event': event,
                'similarity': float(similarity)
            })
        
        # Сортируем
        results.sort(key=lambda x: x['similarity'], reverse=True)
        return results[:top_k]

# ============ 6. ОСНОВНЫЕ ФУНКЦИИ ============
async def parse_real_events(cities=None):
    """Парсинг реальных актуальных событий"""
    if cities is None:
        cities = ['spb', 'msk']
    
    print("="*70)
    print(" ПАРСИНГ РЕАЛЬНЫХ АКТУАЛЬНЫХ СОБЫТИЙ")
    print("="*70)
    print(" События от текущей даты до конца 2025 года")
    print("  С картинками и полными описаниями")
    print("="*70)
    
    all_events = []
    
    for city in cities:
        try:
            parser = RealKudaGoParser2025(city)
            events = await parser.get_real_events()
            
            if events:
                all_events.extend(events)
                
                # Статистика по городу
                city_name = parser.cities.get(city, {}).get('name', city)
                categories = {}
                total_images = 0
                
                for e in events:
                    cat = e['category']
                    categories[cat] = categories.get(cat, 0) + 1
                    total_images += e.get('image_count', 0)
                
                print(f"\n {city_name}:")
                print(f"   Событий: {len(events)}")
                print(f"   Изображений: {total_images}")
                print(f"   Категории:")
                for cat, count in list(categories.items())[:5]:  # Показываем первые 5
                    print(f"     • {cat}: {count}")
                
                # Примеры событий
                print(f"\n   Примеры событий:")
                for i, e in enumerate(events[:3]):
                    print(f"     {i+1}. {e['title'][:60]}...")
                    print(f"        Категория: {e['category']}")
                    if e.get('dates_text'):
                        print(f"        Дата: {e['dates_text'][0]}")
                    print(f"        Цена: {e['price']}")
                    if e.get('place_text'):
                        print(f"        Место: {e['place_text'][:40]}...")
                    if e.get('image_count', 0) > 0:
                        print(f"        Изображений: {e['image_count']}")
                
                # Сохраняем сырые данные
                save_raw_data(events, city)
                
        except Exception as e:
            print(f" Ошибка при парсинге {city}: {e}")
            import traceback
            traceback.print_exc()
    
    return all_events

def save_raw_data(events, city):
    """Сохранение сырых данных"""
    if not events:
        return
    
    data_dir = f"./real_data/{city}"
    os.makedirs(data_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{city}_events_{timestamp}.json"
    filepath = os.path.join(data_dir, filename)
    
    # Создаем копию без эмбеддингов для экономии места
    events_light = []
    for event in events:
        event_copy = event.copy()
        # Удаляем большие поля
        event_copy.pop('embedding_text', None)
        event_copy.pop('description', None)  # Оставляем только короткое описание
        events_light.append(event_copy)
    
    metadata = {
        'city': city,
        'city_name': 'Санкт-Петербург' if city == 'spb' else 'Москва',
        'total_events': len(events),
        'total_images': sum(e.get('image_count', 0) for e in events),
        'parsed_at': datetime.now().isoformat(),
        'year_range': f"{datetime.now().year}-2025",
        'source': 'KudaGo API'
    }
    
    data = {'metadata': metadata, 'events': events_light}
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    print(f" Сырые данные сохранены: {filepath}")

def create_real_vector_database(events):
    """Создание векторной базы для реальных событий"""
    if not events:
        print(" Нет событий для создания БД")
        return None, None
    
    print("\n" + "="*70)
    print(" СОЗДАНИЕ ВЕКТОРНОЙ БАЗЫ ДАННЫХ")
    print("="*70)
    
    db = RealVectorDatabase(folder='real_vector_db')
    db_file = db.save_events(events)
    
    return db, db_file

def test_real_search(db):
    """Тестирование поиска реальных событий"""
    if not db:
        print(" Нет базы данных для тестирования")
        return
    
    print("\n ТЕСТИРУЕМ ПОИСК РЕАЛЬНЫХ СОБЫТИЙ:")
    print("-"*50)
    
    test_cases = [
        ("концерт", None),
        ("выставка искусств", "spb"),
        ("фестиваль", "msk"),
        ("театр", None),
        ("бесплатные мероприятия", None)
    ]
    
    for query, city in test_cases:
        print(f"\n Поиск: '{query}'", end="")
        if city:
            print(f" в городе {city}")
        else:
            print(" во всех городах")
        
        results = db.search_similar(query, top_k=3, city_filter=city)
        
        if results:
            for i, r in enumerate(results):
                e = r['event']
                print(f"  {i+1}. {e['title'][:50]}...")
                print(f"     Категория: {e['category']}")
                print(f"     Город: {e['city_name']}")
                if e.get('dates_text'):
                    print(f"     Дата: {e['dates_text'][0]}")
                print(f"     Цена: {e['price']}")
                if e.get('image_count', 0) > 0:
                    print(f"     Изображений: {e['image_count']}")
                print(f"     Сходство: {r['similarity']:.3f}")
        else:
            print("   Нет результатов")

def generate_real_report(events, db_file):
    """Генерация отчета по реальным событиям"""
    report = f"""# Отчет о парсинге реальных событий

## Общая информация
- Дата парсинга: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- Источник данных: KudaGo API
- Период событий: от текущей даты до конца 2025 года
- Всего событий: {len(events)}

## Статистика по городам
"""
    
    # Статистика по городам
    city_stats = {}
    image_stats = {}
    date_range_stats = {}
    
    for e in events:
        city = e.get('city_name', 'Неизвестно')
        city_stats[city] = city_stats.get(city, 0) + 1
        image_stats[city] = image_stats.get(city, 0) + e.get('image_count', 0)
        
        # Анализ дат
        dates = e.get('dates', [])
        if dates:
            for date_info in dates:
                year = date_info.get('year')
                if year:
                    key = f"{city}_{year}"
                    date_range_stats[key] = date_range_stats.get(key, 0) + 1
    
    for city, count in city_stats.items():
        images = image_stats.get(city, 0)
        report += f"- **{city}**: {count} событий ({images} изображений)\n"
    
    # Статистика по категориям
    report += "\n## Статистика по категориям\n"
    category_stats = {}
    for e in events:
        cat = e.get('category', 'Неизвестно')
        category_stats[cat] = category_stats.get(cat, 0) + 1
    
    for cat, count in sorted(category_stats.items(), key=lambda x: x[1], reverse=True)[:10]:
        report += f"- **{cat}**: {count} событий\n"
    
    # Примеры событий
    report += "\n## Примеры событий\n"
    events_with_images = [e for e in events if e.get('image_count', 0) > 0]
    sample_events = events_with_images[:5] if events_with_images else events[:5]
    
    for i, e in enumerate(sample_events):
        report += f"\n{i+1}. **{e['title']}**\n"
        report += f"   - Город: {e['city_name']}\n"
        report += f"   - Категория: {e['category']}\n"
        if e.get('dates_text'):
            report += f"   - Дата: {e['dates_text'][0]}\n"
        report += f"   - Цена: {e['price']}\n"
        report += f"   - Изображений: {e.get('image_count', 0)}\n"
        if e.get('place_text'):
            report += f"   - Место: {e['place_text'][:50]}...\n"
        if e.get('description'):
            report += f"   - Описание: {e['description'][:100]}...\n"
    
    # Информация о файлах
    report += f"\n## Файлы данных\n"
    report += f"- Векторная база: `{db_file}`\n"
    report += f"- Сырые данные: папки `./real_data/`\n"
    report += f"- Источник: KudaGo Public API v1.4\n"
    
    # Инструкции
    report += f"\n## Примеры использования\n"
    report += f"```python\n"
    report += f"# Загрузка данных\n"
    report += f"import json\n"
    report += f"with open('{db_file}', 'r', encoding='utf-8') as f:\n"
    report += f"    data = json.load(f)\n"
    report += f"print(f'Событий: {{len(data[\"events\"])}}')\n"
    report += f"print(f'Города: {{data[\"metadata\"][\"cities\"]}}')\n"
    report += f"```\n"
    
    # Сохраняем отчет
    report_file = "real_events_report.md"
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(report)
    
    print(f"\n Отчет сохранен: {report_file}")
    
    # Показываем начало отчета
    print(f"\n Начало отчета:")
    print("-" * 50)
    with open(report_file, 'r', encoding='utf-8') as f:
        content = f.read()
        lines = content.split('\n')[:20]
        for line in lines:
            print(line)
    
    return report_file

def show_real_files():
    """Показать созданные файлы"""
    print("\n СОЗДАННЫЕ ФАЙЛЫ:")
    print("="*70)
    
    folders_to_check = ['./real_data', './real_vector_db']
    
    for folder in folders_to_check:
        if os.path.exists(folder):
            print(f"\n Папка: {folder}")
            files = os.listdir(folder)
            if files:
                for file in sorted(files)[-3:]:  # Последние 3 файла
                    filepath = os.path.join(folder, file)
                    if os.path.isfile(filepath):
                        size = os.path.getsize(filepath)
                        print(f"    {file} ({size // 1024} KB)")
        else:
            print(f"\n Папка не создана: {folder}")
    
    # Показываем отчет
    if os.path.exists("real_events_report.md"):
        size = os.path.getsize("real_events_report.md")
        print(f"\n Отчет: real_events_report.md ({size // 1024} KB)")
    
    print(f"\n Текущая директория: {os.getcwd()}")

# ============ 7. ГЛАВНАЯ ФУНКЦИЯ ============
async def main_real():
    """Основная функция для парсинга реальных событий"""
    try:
        print("\n ПАРСИНГ РЕАЛЬНЫХ СОБЫТИЙ")
        print("="*70)
        print(" Цель: получить актуальные события с картинками и описаниями")
        print("Период: от сегодня до конца 2025 года")
        print("  Города: Москва и Санкт-Петербург")
        print("="*70)
        
        # Парсим реальные события
        events = await parse_real_events(['spb', 'msk'])
        
        if not events:
            print("\n  Внимание: не удалось получить события.")
            print("   Возможные причины:")
            print("   1. Нет подключения к интернету")
            print("   2. API KudaGo временно недоступно")
            print("   3. Нет актуальных событий в выбранных категориях")
            return
        
        # Создаем векторную базу
        db, db_file = create_real_vector_database(events)
        
        if db:
            # Тестируем поиск
            test_real_search(db)
            
            # Создаем отчет
            report_file = generate_real_report(events, db_file)
            
            # Показываем файлы
            show_real_files()
            
            print("\n" + "="*70)
            print(" ЗАДАНИЕ ВЫПОЛНЕНО!")
            print("="*70)
            print(f"Всего событий: {len(events)}")
            print(f" Векторная БД: {db_file}")
            print(f" Отчет: {report_file}")
            print(f" Сырые данные: ./real_data/")
            print("="*70)
            
    except Exception as e:
        print(f" Критическая ошибка: {e}")
        import traceback
        traceback.print_exc()

# ============ 8. ЗАПУСК ПРОГРАММЫ ============
if __name__ == "__main__":
    print(" Запуск системы парсинга реальных событий")
    print("="*70)
    print(" Период: текущая дата - конец 2025 года")
    print("  Города: Москва и Санкт-Петербург")
    print("  Данные: события с картинками и описаниями")
    print("="*70)
    
    # Проверяем, в Jupyter ли мы
    try:
        from IPython import get_ipython
        IS_JUPYTER = get_ipython() is not None
    except:
        IS_JUPYTER = False
    
    if IS_JUPYTER:
        print("\n РЕЖИМ JUPYTER NOTEBOOK")
        print("Для запуска парсинга в новой ячейке выполните:")
        print("```python")
        print("await main_real()")
        print("```")
        print("\nИли для тестирования отдельных функций:")
        print("```python")
        print("# Парсинг только Москвы")
        print("parser = RealKudaGoParser2025('msk')")
        print("events = await parser.get_real_events()")
        print()
        print("# Создание БД")
        print("db = RealVectorDatabase()")
        print("db_file = db.save_events(events)")
        print()
        print("# Поиск событий")
        print("results = db.search_similar('концерт')")
        print()
        print("# Просмотр файлов")
        print("show_real_files()")
        print("```")
    else:
        print("\n РЕЖИМ PYCHARM / ТЕРМИНАЛ")
        print("Запускаем парсинг реальных событий...")
        asyncio.run(main_real())

 Проверяем библиотеки...
 Устанавливаем scikit-learn...
 Запуск системы парсинга реальных событий
 Период: текущая дата - конец 2025 года
  Города: Москва и Санкт-Петербург
  Данные: события с картинками и описаниями

 РЕЖИМ JUPYTER NOTEBOOK
Для запуска парсинга в новой ячейке выполните:
```python
await main_real()
```

Или для тестирования отдельных функций:
```python
# Парсинг только Москвы
parser = RealKudaGoParser2025('msk')
events = await parser.get_real_events()

# Создание БД
db = RealVectorDatabase()
db_file = db.save_events(events)

# Поиск событий
results = db.search_similar('концерт')

# Просмотр файлов
show_real_files()
```


In [6]:
await main_real()


 ПАРСИНГ РЕАЛЬНЫХ СОБЫТИЙ
 Цель: получить актуальные события с картинками и описаниями
Период: от сегодня до конца 2025 года
  Города: Москва и Санкт-Петербург
 ПАРСИНГ РЕАЛЬНЫХ АКТУАЛЬНЫХ СОБЫТИЙ
 События от текущей даты до конца 2025 года
  С картинками и полными описаниями

  Парсим РЕАЛЬНЫЕ актуальные события для Санкт-Петербург
 Загружаем Концерты...
     95 актуальных событий
   Обрабатываем 95 событий категории Концерты...
     Обработано 95 событий
 Загружаем Театр...
     81 актуальных событий
   Обрабатываем 81 событий категории Театр...
     Обработано 81 событий
 Загружаем Выставки...
     8 актуальных событий
   Обрабатываем 8 событий категории Выставки...
     Обработано 8 событий
 Загружаем Фестивали...
     9 актуальных событий
   Обрабатываем 9 событий категории Фестивали...
     Обработано 9 событий
 Загружаем Образование...
     28 актуальных событий
   Обрабатываем 28 событий категории Образование...
     Обработано 28 событий
 Загружаем Вечеринки...
     7 актуаль